Description: This is a mock take-home challenge from a pretend company called "Relax, Inc." We're given two .csv files, one which contains records about each signed-up user and one which is a log of individual signins. 

Objective: The challenge is to identify which factors predict future user adoption, where an "adopted user" is defined as a user who has logged into the product on three separate days in at least one seven­day period

Deliverables: A code notebook and a brief writeup

In [1]:
#import basic libaries for data wrangling and eda  
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

plt.style.use('seaborn')

In [2]:
#load the users csv to a pandas df and check it out
users = pd.read_csv('takehome_users.csv', encoding='latin1')
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [3]:
users.shape

(12000, 10)

In [4]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [5]:
users.nunique()

object_id                     12000
creation_time                 11996
name                          11355
email                         11980
creation_source                   5
last_session_creation_time     8821
opted_in_to_mailing_list          2
enabled_for_marketing_drip        2
org_id                          417
invited_by_user_id             2564
dtype: int64

There are 12000 unique "object_id" entries (a unique identifier for each user), but only 8823 recorded "last_session_creation_time"s. We can already see that we have some repeated user names and email addresses, which we'll need to investigate further down the road. Speaking of that creation time, the object type for that is float64 because it's actually a unix timestamp. The creation time feature is of type object. We may need to convert these to dt objects down the line, but for now I'll just take note of them.

In [6]:
#load the user engagement csv to a pandas df and check it out
usage = pd.read_csv('takehome_user_engagement.csv', encoding='latin1')
usage.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [7]:
usage.shape

(207917, 3)

In [8]:
usage.nunique()

time_stamp    207220
user_id         8823
visited            1
dtype: int64

In [9]:
usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [10]:
#check for duplicates
usage.drop_duplicates(inplace=True)
usage.shape

(207917, 3)

We have timestamp data for 207,917 distinct logins from 8823 distinct users. The timestamp data is of type object and will need to be converted to datetime to do any calculations with it.

Lets go ahead and do that now.


In [11]:
import datetime as dt

usage['time_stamp'] = pd.to_datetime(usage['time_stamp'])

usage.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time_stamp  207917 non-null  datetime64[ns]
 1   user_id     207917 non-null  int64         
 2   visited     207917 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 6.3 MB


My plan of attack here is to define a function which, for each user in users, checks in usage whether that user has had 3 logins in a 7 day period, returning 1 if they are active and 0 if they are not and adding that value into a new column "active" in the users df.

In [12]:
def activity(id):
    df = usage[usage['user_id'] == id]
    if len(df) < 3:
        return 0
    else: 
        for i in range(len(df)-2):
            if (df['time_stamp'].iloc[i+2] - df['time_stamp'].iloc[i]).days <=7:
                return 1
        return 0

In [13]:
#create a list which holds all the users who have logged in
usage_users = list(usage['user_id'].unique())

#create an empty list to hold active status
active_status = []

#find status of each user who has logged in
for user in usage_users:
    status = activity(id=user)
    active_status.append(status)

#create a 3rd df that holds active status for each user
active_users = pd.DataFrame({'user_id' : usage_users, 'active' : active_status})

active_users.head()

,user_id,active
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0


In [14]:
print(active_users.active.value_counts())

print(active_users.active.value_counts(normalize=True))

0    7167
1    1656
Name: active, dtype: int64
0    0.812309
1    0.187691
Name: active, dtype: float64


So, of the users we have login data for, only 18.7% of them are active users.

Let's wrap up this part of the work by joining the active users table to the users df.

In [15]:
#rename 'object_id' to 'user_id' to prepare for the join
users.rename(columns = {'object_id' : 'user_id'}, inplace=True)

users = pd.merge(users, active_users, on='user_id', how='outer')

users.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,active
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0.0


In [16]:
#time for some clean up

#convert times to dt objects
users['creation_time'] = pd.to_datetime(users['creation_time'])
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'], unit='s')

#fill NaN values in 'active' and 'invited_by_user_id' with 0
users['active'].fillna(0,inplace=True)
users['invited_by_user_id'].fillna(0,inplace=True)

#cast active and invited_by_user_id columns to type int
users['active'] = users['active'].astype(int)
users['invited_by_user_id'] = users['invited_by_user_id'].astype(int)

#standardize name and email fields to be all lowercase
users['name'] = users['name'].str.lower()
users['email'] = users['email'].str.lower()

#sort df so that active users are first:
users.sort_values(['active'], inplace=True, ascending=False)



In [17]:
#check duplicate records based on matching name and email address and org_id
old_users = users
duplicate_name_and_email = users.drop_duplicates(['email', 'name'], keep='first')
duplicate_name_email_and_orgid = users.drop_duplicates(['email', 'name', 'org_id'], keep='first')
print('there are {} records with same name and email address'.format(len(old_users) - len(duplicate_name_and_email)))
print('there are {} records with same name and email address and org_id'.format(len(old_users) - len(duplicate_name_email_and_orgid)))


there are 19 records with same name and email address
there are 0 records with same name and email address and org_id


So, that's interesting, there are a handful of duplicate records with the same name and email address, however when we take into account the organization id number, there are no duplicate entries. Because an association with a different organization could be a factor in a user's active status, we won't drop any of these entries.

In [18]:
#drop duplicate records based on matching name and email address, keep first record (which in this case would be the active record)
users = users.drop_duplicates(['email', 'name'], keep='first')

In [19]:
users.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,active
3221,3222,2013-03-31 06:14:26,pope alex,alexpope@gmail.com,SIGNUP_GOOGLE_AUTH,2013-04-05 06:14:26,0,0,31,0,1
4834,4835,2012-06-17 03:07:30,lloyd cerys,ceryslloyd@jourrapide.com,SIGNUP,2014-06-04 03:07:30,1,0,251,0,1
10232,10233,2013-11-28 18:02:31,karlsen carl,nfmtiitt@njesf.com,ORG_INVITE,2014-06-03 18:02:31,0,0,103,2017,1
7759,7760,2013-10-03 21:38:09,de satg eva,evadesatg@yahoo.com,ORG_INVITE,2014-05-18 21:38:09,0,1,277,2120,1
2041,2042,2012-06-10 00:11:04,silva davi,davirodriguessilva@gmail.com,SIGNUP_GOOGLE_AUTH,2014-06-04 00:11:04,0,0,129,0,1


Now, to identify which factors predict future user adoption, I'm going to start with a heatmap

In [20]:
print(users['invited_by_user_id'].astype(int).head())

3221        0
4834        0
10232    2017
7759     2120
2041        0
Name: invited_by_user_id, dtype: int64
